In [1]:
import pandas as pd
import numpy as np

import gc, os, time

In [2]:
prev = pd.read_csv('DATA/previous_application.csv/previous_application.csv')
ins = pd.read_csv('DATA/installments_payments.csv/installments_payments.csv')

In [14]:
cat_cols = prev.select_dtypes('object').columns
cat_cols

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [4]:
prev['NAME_CASH_LOAN_PURPOSE'].unique()

array(['XAP', 'XNA', 'Repairs', 'Everyday expenses', 'Car repairs',
       'Building a house or an annex', 'Other', 'Journey',
       'Purchase of electronic equipment', 'Medicine',
       'Payments on other loans', 'Urgent needs', 'Buying a used car',
       'Buying a new car', 'Buying a holiday home / land', 'Education',
       'Buying a home', 'Furniture', 'Buying a garage',
       'Business development', 'Wedding / gift / holiday', 'Hobby',
       'Gasification / water supply', 'Refusal to name the goal',
       'Money for a third person'], dtype=object)

In [13]:
# One hot encoder
one_hot_cols = ['NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_PORTFOLIO', 'CHANNEL_TYPE', 'NAME_YIELD_GROUP']
prev, categorical_columns = one_hot_encoder(prev, one_hot_cols, nan_as_category= False)

In [16]:
categorical_columns

['NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Consumer loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'NAME_CONTRACT_TYPE_XNA',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Canceled',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Unused offer',
 'NAME_PAYMENT_TYPE_Cash through the bank',
 'NAME_PAYMENT_TYPE_Cashless from the account of the employer',
 'NAME_PAYMENT_TYPE_Non-cash from your account',
 'NAME_PAYMENT_TYPE_XNA',
 'NAME_TYPE_SUITE_Children',
 'NAME_TYPE_SUITE_Family',
 'NAME_TYPE_SUITE_Group of people',
 'NAME_TYPE_SUITE_Other_A',
 'NAME_TYPE_SUITE_Other_B',
 'NAME_TYPE_SUITE_Spouse, partner',
 'NAME_TYPE_SUITE_Unaccompanied',
 'NAME_CLIENT_TYPE_New',
 'NAME_CLIENT_TYPE_Refreshed',
 'NAME_CLIENT_TYPE_Repeater',
 'NAME_CLIENT_TYPE_XNA',
 'NAME_PORTFOLIO_Cards',
 'NAME_PORTFOLIO_Cars',
 'NAME_PORTFOLIO_Cash',
 'NAME_PORTFOLIO_POS',
 'NAME_PORTFOLIO_XNA',
 'CHANNEL_TYPE_AP+ (Cash loan)',
 'CHANNEL_TYPE_Car dealer',
 'CHANNEL_TYPE_Channel of corpo

Ratio and Difference

In [7]:
prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
prev['APPLICATION_TO_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
prev['DOWN_PAYMENT_TO_CREDIT_RATIO'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']
prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']


In [30]:
## Last due : outliner 365243 that mean is not complete

aproved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
active = aproved[aproved['DAYS_LAST_DUE'] == 365243]

# Find how much already pay in active loans ( using installments data)
active_pay = ins[ins['SK_ID_PREV'].isin(active['SK_ID_PREV'])]
active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
active_pay_agg.reset_index(inplace =True)
active_pay_agg


,SK_ID_PREV,AMT_INSTALMENT,AMT_PAYMENT
0,1000003,14854.050,14854.050
1,1000007,56234.025,56234.025
2,1000018,35257.905,35257.905
3,1000026,93430.620,93430.620
4,1000030,43398.495,43398.495
...,...,...,...
172578,2843487,57438.360,57438.360
172579,2843491,254219.850,254219.850
172580,2843493,79887.060,77637.060
172581,2843496,312334.605,304333.155


In [31]:
# Active pay: diff from what was payed and installments
active_pay_agg['INSTALMEN_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']

# merge with active
active = active.merge(active_pay_agg, on= 'SK_ID_PREV', how= 'left')
active['REMAINING_DEBT'] = active['AMT_CREDIT'] - active['AMT_PAYMENT']
active['PAYMENT_TO_CREDIT_RATIO'] = active['AMT_PAYMENT'] / active['AMT_CREDIT']

In [38]:
active_agg = group(active, 'PREV_ACTIVE_', PREVIOUS_ACTIVE_AGG)

In [37]:
PREVIOUS_ACTIVE_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['max', 'sum', 'mean'],
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['sum', 'max'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'mean', 'max'],
    'CNT_PAYMENT': ['mean', 'sum'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    'DAYS_FIRST_DUE' :['min', 'max', 'mean'],

    # engineered features
    'AMT_PAYMENT':['sum'],
    'INSTALMEN_PAYMENT_DIFF': ['mean', 'max'],
    'REMAINING_DEBT': ['max', 'mean', 'sum'],
    'PAYMENT_TO_CREDIT_RATIO': ['mean'],
}

In [41]:
active_agg['TOTAL_PAYMENT_TO_CREDIT_RATIO'] = active_agg['PREV_ACTIVE_AMT_PAYMENT_SUM'] / active_agg['PREV_ACTIVE_AMT_CREDIT_SUM']

Outliner

In [42]:
prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

Aggregate prev

In [45]:
categorical_columns = {col: ['mean'] for col in categorical_columns}
prev_agg = group(prev, 'PREV_', {**PREVIOUS_AGG, **categorical_columns})

In [44]:
PREVIOUS_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['max'],

    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'var'],
    'APPLICATION_TO_CREDIT_RATIO': ['mean', 'max', 'min', 'var'],
    'APPLICATION_CREDIT_DIFF': ['mean', 'max', 'min'],
    'DOWN_PAYMENT_TO_CREDIT_RATIO': ['mean'],
}

In [47]:
# Merge active loan df on prev_agg
prev_agg = prev_agg.merge(active_agg, on= 'SK_ID_CURR', how= 'left')

In [54]:
# Aggregate for approved and refused
prev_agg = group_and_merge(aproved, prev_agg, 'APPROVED_', PREVIOUS_APPROVED_AGG)
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
prev_agg = group_and_merge(refused, prev_agg, 'REFUSED_', PREVIOUS_REFUSED_AGG)

In [51]:
PREVIOUS_APPROVED_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max'],
    'AMT_GOODS_PRICE': ['max'],
    'HOUR_APPR_PROCESS_START': ['min', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['mean'],

    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'var'],
    'APPLICATION_TO_CREDIT_RATIO': ['mean', 'max', 'min', 'var'],
    'APPLICATION_CREDIT_DIFF': ['mean', 'max', 'min'],
    'DOWN_PAYMENT_TO_CREDIT_RATIO': ['mean'],
}

In [52]:
PREVIOUS_REFUSED_AGG = {
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['min', 'max'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],

    'APPLICATION_TO_CREDIT_RATIO': ['mean', 'max', 'min', 'var'],
    'APPLICATION_CREDIT_DIFF': ['mean', 'max', 'min'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

In [60]:
for time in [12,24]:
    time_df = prev[prev['DAYS_DECISION'] >= -30*time]
    prefix = 'PREV_LAST_{}M_'.format(time)
    prev_agg = group_and_merge(time_df, prev_agg, prefix, PREVIOUS_TIME_AGG)

In [7]:
(prev['DAYS_DECISION'] / (-30)).describe()

count    1.670214e+06
mean     2.935599e+01
std      2.596999e+01
min      3.333333e-02
25%      9.333333e+00
50%      1.936667e+01
75%      4.333333e+01
max      9.740000e+01
Name: DAYS_DECISION, dtype: float64

In [59]:
PREVIOUS_TIME_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    
    'APPLICATION_TO_CREDIT_RATIO': ['mean', 'max', 'min', 'var'],
    'APPLICATION_CREDIT_DIFF': ['mean', 'max', 'min'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

In [61]:
prev_agg.to_parquet('TO_TRAIN/prev.gzip', compression= 'gzip', index= False)

In [62]:
prev_agg

,SK_ID_CURR,PREV_SK_ID_PREV_NUNIQUE,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_HOUR_APPR_PROCESS_START_MIN,PREV_HOUR_APPR_PROCESS_START_MAX,PREV_HOUR_APPR_PROCESS_START_MEAN,...,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MEAN,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MAX,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MIN,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_VAR,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MEAN,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MAX,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MIN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Cash loans_MEAN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Revolving loans_MEAN
0,100001,1,3951.000,3951.000,3951.000000,2520.0,2520.00,13,13,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,1,9251.775,9251.775,9251.775000,0.0,0.00,9,9,9.000000,...,1.000000,1.00000,1.000000,NaN,0.0,0.0,0.0,1.000000,0.0,0.000000
2,100003,3,6737.310,98356.995,56553.990000,6885.0,3442.50,12,17,14.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,1,5357.250,5357.250,5357.250000,4860.0,4860.00,5,5,5.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,2,4813.200,4813.200,4813.200000,4464.0,4464.00,10,11,10.500000,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,1,6605.910,6605.910,6605.910000,0.0,0.00,17,17,17.000000,...,1.000000,1.00000,1.000000,NaN,0.0,0.0,0.0,1.000000,0.0,0.000000
338853,456252,1,10074.465,10074.465,10074.465000,3456.0,3456.00,10,10,10.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338854,456253,2,3973.095,5567.715,4770.405000,5913.0,4403.25,11,12,11.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338855,456254,2,2296.440,19065.825,10681.132500,0.0,0.00,12,18,15.000000,...,0.891418,0.90448,0.878356,0.000341,-13122.0,-2610.0,-23634.0,1.000000,0.0,0.000000


In [5]:
# Function to aggregate numeric columns 
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

# Function to merge numeric and categorical after aggregate
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

In [12]:
## One hot encoder

def one_hot_encoder(df, categorical_columns = None, nan_as_category = True):

    original_columns = list(df.columns)
    
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

    # One hot encoder
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)

    # New columns created by get_dummies
    categorical_columns = [col for col in df.columns if col not in original_columns]

    return df, categorical_columns

In [6]:
prev_agg = pd.read_parquet('TO_TRAIN/prev.gzip')
prev = pd.read_csv('DATA/previous_application.csv/previous_application.csv')

In [8]:
prev_agg.head()

,SK_ID_CURR,PREV_SK_ID_PREV_NUNIQUE,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_HOUR_APPR_PROCESS_START_MIN,PREV_HOUR_APPR_PROCESS_START_MAX,PREV_HOUR_APPR_PROCESS_START_MEAN,...,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MEAN,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MAX,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_MIN,PREV_LAST_24M_APPLICATION_TO_CREDIT_RATIO_VAR,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MEAN,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MAX,PREV_LAST_24M_APPLICATION_CREDIT_DIFF_MIN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Cash loans_MEAN,PREV_LAST_24M_NAME_CONTRACT_TYPE_Revolving loans_MEAN
0,100001,1,3951.000,3951.000,3951.000,2520.0,2520.0,13,13,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,1,9251.775,9251.775,9251.775,0.0,0.0,9,9,9.000000,...,1.0,1.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0
2,100003,3,6737.310,98356.995,56553.990,6885.0,3442.5,12,17,14.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,1,5357.250,5357.250,5357.250,4860.0,4860.0,5,5,5.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,2,4813.200,4813.200,4813.200,4464.0,4464.0,10,11,10.500000,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
## Get ids of late payment
ins = pd.read_csv('DATA/installments_payments.csv/installments_payments.csv')

ins['LATE_PAYMENT'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
late_ids = ins[ins['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()
late_ids

array([2714724, 2329072, 1840786, ..., 2429423, 1493367, 1635645],
      dtype=int64)

In [14]:
tmp = group_and_merge(prev[prev['SK_ID_PREV'].isin(late_ids)], prev_agg, 'PREV_LATE', PREVIOUS_LATE_AGG)

In [9]:
PREVIOUS_LATE_AGG = {
    'DAYS_LAST_DUE_1ST_VERSION' : ['mean', 'max', 'min'],
    'DAYS_LAST_DUE': ['mean', 'max', 'min'],
    'DAYS_DECISION': ['mean', 'max', 'min'],

    'DOWN_PAYMENT_TO_CREDIT_RATIO': ['max', 'min'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

In [16]:
tmp.to_parquet('TO_TRAIN/prev.gzip', compression= 'gzip', index= False)